## Lập trình xử lý dữ liệu - Nhóm 7: Đánh đâu lỗ đó
### Notebook Lấy dữ liệu và Tiền xử lý dữ liệu

### Dữ liệu chứng khoán
---

#### Khai báo thư viện

In [1]:
from vnstock3 import Vnstock
import pandas as pd
import json

stock = Vnstock().stock(source='VCI')
df_list_stock = pd.DataFrame(stock.listing.symbols_by_industries())
df_list_post = pd.read_csv('cleaned_posts.csv')

Phiên bản Vnstock 3.1.0 đã có mặt, vui lòng cập nhật với câu lệnh : `pip install vnstock3 --upgrade`.
Lịch sử phiên bản: https://vnstocks.com/docs/tai-lieu/lich-su-phien-ban
Phiên bản hiện tại 0.3.0.9

2024-11-12 11:24:48 - vnstock3.common.vnstock - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2024-11-12 11:24:48 - vnstock3.common.data.data_explorer - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2024-11-12 11:24:48 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


#### Hàm lấy dữ liệu cụ thể
Những hàm sau lấy các thông tin về công ty đại diện cho các mã chứng khoán, lấy từ các nguồn nhờ thư viện `vnstock3`:
- `get_info_industry`: Lấy thông tin về tên công ty, ngành của công ty
- `get_info_overview`: Lấy thông tin về số lượng cổ đông, số lượng nhân viên, ...
- `get_info_finance`: Lấy thông tin về tài chính của công ty, các chỉ số...

In [25]:
def get_info_industry(symbol: str):
    # Lấy hàng chứa thông tin của mã cổ phiếu cần tìm
    df_row = df_list_stock[df_list_stock['symbol'] == symbol]
    df_row = df_row[['symbol', 'organ_name', 'icb_code2', 'icb_name2']]
    df_row.columns = ['symbol', 'name', 'indus_code', 'indus_name']

    return df_row

get_info_industry('VCI')

,symbol,name,indus_code,indus_name
1460,VCI,Công ty Cổ phần Chứng khoán Vietcap,8700,Dịch vụ tài chính


In [50]:
def get_info_overview(symbol: str):
    # Lấy thông tin tổng quan của mã cổ phiếu cần tìm, từ TCBS
    df_info_ovw = pd.DataFrame(Vnstock(show_log=False).stock(symbol=symbol, source='TCBS').company.overview())
    df_info_ovw['symbol'] = symbol
    df_info_ovw = df_info_ovw[['symbol', 'exchange', 'no_shareholders', 'no_employees', 'stock_rating']]

    # Fill NaN thành -1
    df_info_ovw = df_info_ovw.fillna(-1).infer_objects(copy=False)

    # Đổi tên cột
    df_info_ovw.columns = ['symbol', 'exc', 'm_stholder', 'm_employee', 'r_tcbsrating']
    
    return df_info_ovw

get_info_overview('VCI')


,symbol,exc,m_stholder,m_employee,r_tcbsrating
0,VCI,HOSE,15669,386,2.3


In [56]:
def get_info_finance(symbol: str):
    # Thử lấy thông tin tài chính từ VCI
    df_row = pd.DataFrame()
    try:
        df_row = pd.DataFrame(Vnstock(show_log=False).stock(symbol=symbol, source='VCI').finance.ratio(period='year', lang='vi'))
    except:
        pass
    
    df_info_finance = pd.DataFrame()

    # Lấy các chỉ tiêu cơ bản trong năm gần nhất (2023)
    df_row = df_row.head(1)

    LIST_KEYS = [(('Meta', 'CP')                                      , 'symbol' ), 
                 (('Meta', 'Năm')                                     , 'year'   ),
                 (('Chỉ tiêu định giá', 'Vốn hóa (Tỷ đồng)')          , 'm_cap'  ),     # m_cap
                 (('Chỉ tiêu định giá', 'Số CP lưu hành (Triệu CP)')  , 'm_share'),     # m_share
                 (('Chỉ tiêu định giá', 'P/B')                        , 'r_pb'   ),     # r_pb      = giá thị trường / BVPS
                 (('Chỉ tiêu định giá', 'P/E')                        , 'r_pe'   ),     # r_pe      = giá thị trường / lợi nhuận sau thuế
                 (('Chỉ tiêu định giá', 'EPS (VND)')                  , 'r_eps'  ),     # r_eps     = lợi nhuận sau thuế / số CP lưu hành
                 (('Chỉ tiêu định giá', 'BVPS (VND)')                 , 'r_bvps' ),     # r_bvps    = vốn chủ sở hữu / số CP lưu hành
                 (('Chỉ tiêu khả năng sinh lợi', 'ROE (%)')           , 'p_roe'  ),     # p_roe     = lợi nhuận sau thuế / vốn chủ sở hữu
                 (('Chỉ tiêu khả năng sinh lợi', 'Tỷ suất cổ tức (%)'), 'p_div'  ),     # p_div     = Tỷ suất cổ tức = cổ tức / giá thị trường
                ]

    # Kiểm tra xem các chỉ tiêu có trong df_info_finance không
    for key, col_name in LIST_KEYS:
        if key not in df_row.columns:
            df_row[key] = 0

        # Đổi tên cột thanh col_name
        df_info_finance[col_name] = df_row[key]

    return df_info_finance

get_info_finance('VCI')

,symbol,year,m_cap,m_share,r_pb,r_pe,r_eps,r_bvps,p_roe,p_div
0,VCI,2023,15093750000000,437500000,2.047657,30.6843,1124.353507,16848.52459,0.070948,0.0


In [57]:
# Lấy toàn bộ mã cổ phiếu trong list_stock
list_symbol = df_list_stock['symbol'].tolist()
df_info = pd.DataFrame()

for i, symbol in enumerate(list_symbol):
    # Lấy thông tin của mã cổ phiếu ở 3 hàm
    df_info_industry = get_info_industry(symbol)
    df_info_ovw = get_info_overview(symbol)
    df_info_finance = get_info_finance(symbol)

    # Gộp
    df_info_temp = pd.merge(df_info_industry, df_info_ovw, on='symbol', how='inner')
    df_info_temp = pd.merge(df_info_temp, df_info_finance, on='symbol', how='inner')

    df_info = pd.concat([df_info, df_info_temp], ignore_index=True)

    # Lưu vào file CSV
    df_info_temp.to_csv('list_stock.csv', index=False, encoding='utf-8', header=(False if i > 0 else True), mode='a')
    print(f'Processed {symbol} ({i+1}/{len(list_symbol)})!', end='\r')

print(f'\nProcessed {len(df_info)} stocks, end of data')

df_info = df_info.reset_index(drop=True)
df_info.tail(20)

Processed XMC (1532/1532)!
Processed 1528 stocks, end of data


,symbol,name,indus_code,indus_name,exc,m_stholder,m_employee,r_tcbsrating,year,m_cap,m_share,r_pb,r_pe,r_eps,r_bvps,p_roe,p_div
1508,X77,Công ty Cổ phần Thành An 77,2300,Xây dựng và Vật liệu,UPCOM,183,2,-1.0,2023,4.022632e+08,1340877.0,0.000000,4.136201,72.530346,0.000000,-0.000508,0.000000
1509,DLT,Công ty Cổ phần Du lịch và Thương mại - Vinacomin,5700,Du lịch và Giải trí,UPCOM,200,605,-1.0,2023,1.550035e+10,2500056.0,0.262400,5.080386,1987.025164,23628.046929,0.084106,0.161290
1510,WTC,Công ty Cổ phần Vận tải thủy - Vinacomin,2700,Hàng & Dịch vụ Công nghiệp,UPCOM,511,208,2.3,2023,1.150000e+11,10000000.0,0.508799,7.639684,1615.297791,22602.224455,0.071419,0.104348
1511,YBC,Công ty Cổ phần Xi măng và Khoáng sản Yên Bái,2300,Xây dựng và Vật liệu,UPCOM,250,331,-1.0,2023,9.912000e+10,11800000.0,0.798459,11.838954,709.522160,10520.262879,0.070018,0.000000
1512,VTV,Công ty Cổ phần Năng lượng và Môi trường VICEM,2300,Xây dựng và Vật liệu,HNX,0,199,2.4,2023,1.528791e+11,31199825.0,0.371234,341.064444,169.740254,13199.226256,0.012641,0.020408
1513,VTX,Công ty Cổ phần Vận tải Đa phương thức VIETRAN...,2700,Hàng & Dịch vụ Công nghiệp,UPCOM,536,141,2.4,2023,1.677786e+11,20972321.0,0.884538,-2.149692,-3721.463094,9044.273511,-0.341205,0.000000
1514,YBM,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái,1700,Tài nguyên Cơ bản,HOSE,813,223,-1.0,2023,1.572987e+11,14299880.0,0.868799,11.734295,937.423187,12661.149644,0.075341,0.045455
1515,VXB,Công ty Cổ phần Vật liệu xây dựng Bến Tre,2300,Xây dựng và Vật liệu,UPCOM,210,22,-1.0,2022,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1516,CLH,Công ty Cổ phần Xi măng La Hiên VVMI,2300,Xây dựng và Vật liệu,HNX,543,529,3.4,2023,2.628000e+11,12000000.0,1.335775,8.352975,3926.513575,16394.982041,0.235593,0.118721
1517,WSB,Công ty Cổ phần Bia Sài Gòn - Miền Tây,3500,Thực phẩm và đồ uống,UPCOM,789,201,2.7,2023,7.421100e+11,14500000.0,0.971923,9.450694,5753.874760,52658.502164,0.112670,0.078156


In [2]:
# Danh sách ngành
df_list_indus = df_list_stock[['icb_code2', 'icb_name2']].drop_duplicates().reset_index(drop=True)
df_list_indus

,icb_code2,icb_name2
0,8300,Ngân hàng
1,3500,Thực phẩm và đồ uống
2,2300,Xây dựng và Vật liệu
3,1300,Hóa chất
4,7500,"Điện, nước & xăng dầu khí đốt"
5,9500,Công nghệ Thông tin
6,5500,Truyền thông
7,2700,Hàng & Dịch vụ Công nghiệp
8,4500,Y tế
9,5700,Du lịch và Giải trí


#### Hàm lấy giá lịch sử

Hàm này sẽ lấy giá lịch sử của các mã được đề cập đến trong những bài viết trong file `cleaned_posts.csv`.
- Khung thời gian lấy sẽ là 30 phút, trong trường hợp không có đủ dữ liệu thì dùng khung 1 ngày.
- Bắt đầu lấy dữ liệu từ ngày `01/9/2024` cho đến `08/11/2024`.

In [11]:
def process_symbol(cmponent, symb, interval='1D'):
    try:
        df = pd.DataFrame(cmponent.quote.history(symbol=symb, start='2024-09-01', end='2024-11-08', interval = interval))
        if len(df) == 0:
            raise Exception('No data')
        
        # Xóa các dòng  không cần thiết
        df = df.dropna()

        # Lưu vào file CSV
        df.to_csv('price/' + symb + '.csv', index=False)
        return 1
    except Exception as e:
        print(f'\nError processing {symb}: {e}')
        return 0

In [8]:
# Lấy danh sách mã cổ phiếu được đề cập
set_taggedSymbols = set()

for post in df_list_post['taggedSymbols']:
    symb_array = json.loads(post)
    for symb in symb_array:
        set_taggedSymbols.add(symb['symb'])

list_taggedSymbols = list(set_taggedSymbols)
list_taggedSymbols.sort()

print(list_taggedSymbols)

['$AAVE', '$ADA', '$AGLD', '$ALGO', '$ALT', '$APT', '$ARB', '$ATOM', '$BCH', '$BNB', '$BNX', '$BTC', '$CAKE', '$CELO', '$CKB', '$DGB', '$DODO', '$DOGE', '$DOT', '$EGLD', '$ETC', '$ETH', '$FET', '$FIDA', '$FORTH', '$FTM', '$GALA', '$HIFI', '$LINK', '$LRC', '$LUNC', '$MAGIC', '$MKR', '$NEAR', '$OG', '$OM', '$OP', '$PAXG', '$PEOPLE', '$PEPE', '$PIXEL', '$RARE', '$RONIN', '$RUNE', '$SHIB', '$SOL', '$TIA', '$TROY', '$TRU', '$TRX', '$TWT', '$USDC', '$USDP', '$VIC', '$WBTC', '$WLD', '$XAI', '$XRP', '$ZEC', 'A32', 'AAA', 'AAH', 'AAM', 'AAS', 'AAT', 'AAV', 'ABB', 'ABC', 'ABI', 'ABS', 'ABT', 'ABW', 'ACB', 'ACC', 'ACE', 'ACG', 'ACL', 'ACM', 'ACV', 'ADG', 'ADP', 'ADS', 'AFX', 'AGF', 'AGG', 'AGM', 'AGP', 'AGR', 'AGX', 'AIC', 'ALT', 'ALV', 'AMD', 'AMS', 'AMV', 'ANT', 'ANV', 'APF', 'APG', 'APH', 'API', 'APP', 'APS', 'ARM', 'ART', 'ASG', 'ASM', 'ASP', 'AST', 'ATB', 'ATG', 'AUD-USD', 'AVC', 'AVF', 'AVG', 'BAB', 'BAF', 'BAL', 'BAX', 'BBC', 'BBH', 'BBM', 'BBT', 'BCA', 'BCB', 'BCC', 'BCE', 'BCF', 'BCG', '

In [12]:
# Kiểm tra xem thư mục price có tồn tại không, nếu không thì tạo mới
import os
if not os.path.exists('price'):
    os.makedirs('price')

count_processed = 0

for i, symb in enumerate(list_taggedSymbols):

    # Phân loại mã cổ phiếu
    if symb[0] == '$':  # Tiền điện tử (crypto)
        crypto = Vnstock().crypto(source='MSN')
        count_processed += process_symbol(crypto, symb[1:])
    elif symb[0] == '^':  # Chỉ số (index)
        index = Vnstock().world_index(source='MSN')
        count_processed += process_symbol(index, symb[1:])
    elif '-' in symb:     # Tỷ giá (forex)
        forex = Vnstock().fx(source='MSN')
        count_processed += process_symbol(forex, symb.replace('-', ''))
    elif len(symb) > 3 and symb[0] == 'C': # Chứng quyền (warrant), bỏ qua
        continue
    elif len(symb) > 3 and (symb[-2::] == '24' or symb[-2::] == '25'): # Hợp đồng tương lai (future), bỏ qua
        continue
    elif '=' in symb or '.' in symb:     # Một số mã cổ phiếu khác, bỏ qua
        continue
    else:
        count_processed += process_symbol(stock, symb, '30m') # Mã cổ phiếu thông thường, lấy dữ liệu 30 phút
    print(f'Processed {symb} ({i+1}/{len(list_taggedSymbols)}), saved to price/{symb}.csv', end='\r')

print(f'\nProcessed {count_processed} stocks, end of data')



Error processing AAVE: 'AAVE'


2024-11-09 03:04:25 - vnstock3.common.data.data_explorer - INFO - Chuyển đổi mã chứng khoán ADA sang mã chứng khoán MSN: c2114


Processed $ADA (2/1481), saved to price/$ADA.csv
Error processing AGLD: 'AGLD'
Processed $AGLD (3/1481), saved to price/$AGLD.csv
Error processing ALGO: 'ALGO'
Processed $ALGO (4/1481), saved to price/$ALGO.csv
Error processing ALT: 'ALT'
Processed $ALT (5/1481), saved to price/$ALT.csv
Error processing APT: 'APT'
Processed $APT (6/1481), saved to price/$APT.csv
Error processing ARB: 'ARB'
Processed $ARB (7/1481), saved to price/$ARB.csv
Error processing ATOM: 'ATOM'
Processed $ATOM (8/1481), saved to price/$ATOM.csv
Error processing BCH: 'BCH'


2024-11-09 03:04:29 - vnstock3.common.data.data_explorer - INFO - Chuyển đổi mã chứng khoán BNB sang mã chứng khoán MSN: c2113


Processed $BNB (10/1481), saved to price/$BNB.csv
Error processing BNX: 'BNX'


...

Processed ^HSI (1475/1481), saved to price/^HSI.csv
Error processing IXIC: 'IXIC'
Processed ^IXIC (1476/1481), saved to price/^IXIC.csv
Error processing KS11: 'KS11'


2024-11-09 03:13:43 - vnstock3.common.data.data_explorer - INFO - Chuyển đổi mã chứng khoán N225 sang mã chứng khoán MSN: a9j7bh


2024-11-09 03:13:44 - vnstock3.common.data.data_explorer - INFO - Chuyển đổi mã chứng khoán NYA sang mã chứng khoán MSN: a74pqh


2024-11-09 03:13:45 - vnstock3.common.data.data_explorer - INFO - Chuyển đổi mã chứng khoán RUT sang mã chứng khoán MSN: b9v42w


Processed ^RUT (1480/1481), saved to price/^RUT.csv
Error processing SSE: 'SSE'
Processed ^SSE (1481/1481), saved to price/^SSE.csv
Processed 1154 stocks, end of data
